# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

### Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

###About MNIST:
The problem we are trying to solve here is to classify grayscale images of handwritten digits (28 pixels by 28 pixels), into their 10 categories (0 to 9). The dataset we will use is the MNIST dataset, a classic dataset in the machine learning community, which has been around for almost as long as the field itself and has been very intensively studied. It's a set of 60,000 training images, plus 10,000 test images, assembled by the National Institute of Standards and Technology (the NIST in MNIST) in the 1980s. 

The MNIST dataset comes pre-loaded in Keras, in the form of a set of four Numpy arrays:

#### Import the mnist dataset from keras datasets

In [0]:
from keras.datasets import mnist

In [0]:
import numpy as np
import datetime

np.random.seed(42) 
import keras
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout

## Settings

In [0]:
now = datetime.datetime.now

batch_size = 128
nb_classes = 5
nb_epoch = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

In [74]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

#### Creating two datasets one with digits below 5 and one with 5 and above

In [0]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
x_test_gt5 = x_test[y_test >= 5]         # np_utils.to_categorical
y_test_gt5 = y_test[y_test >= 5] - 5

### Check 

Verify shapes of x_train, y_train, x_test and y_test for both the datasets with the below given shapes.

In [76]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(30596, 28, 28)
(30596,)
(5139, 28, 28)
(5139,)


In [77]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


### Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST
### Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [0]:
x_train_conv = x_train_lt5.reshape(x_train_lt5.shape[0], 28, 28, 1)
x_test_conv = x_test_lt5.reshape(x_test_lt5.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

### Change into float32 datatype and Normalize x_train and x_test by dividing it by 255.0

In [0]:
x_train_conv =  x_train_conv.astype("float32") / 255
x_test_conv = x_test_conv.astype("float32") / 255

### Check

Verify the shapes of the X_train and X_test with the shapes given below.

In [80]:
print('X_train shape:', x_train_lt5.shape)
print('X_test shape:', x_test_lt5.shape)

X_train shape: (30596, 28, 28)
X_test shape: (5139, 28, 28)


### Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train_class = keras.utils.to_categorical(y_train_lt5, 5)
y_test_class = keras.utils.to_categorical(y_test_lt5, 5)

### Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [0]:
cnn_1 = Sequential()
cnn_1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1), name='conv_1'))
cnn_1.add(MaxPooling2D(pool_size=(2, 2)))
cnn_1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1), name='conv_2'))
cnn_1.add(MaxPooling2D(pool_size=(2, 2)))
cnn_1.add(Dropout(0.25))

### Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [0]:
cnn_1.add(Flatten())
cnn_1.add(Dense(128, activation='relu', name='dense_1'))
cnn_1.add(Dense(5, activation='softmax', name='dense_2'))

### Print the training and test accuracy for 5 epochs

In [84]:
cnn_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               102528    
__________

In [0]:
cnn_1.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [86]:
# fits the model on batches with real-time data augmentation:
model_1 = cnn_1.fit(x_train_conv, y_train_class, 
                     validation_data=(x_test_conv, y_test_class),        
                     batch_size=512, epochs=5)

Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 1s 36us/step - loss: 0.3762 - acc: 0.8953 - val_loss: 0.0593 - val_acc: 0.9805
Epoch 2/5
30596/30596 [==============================] - 1s 20us/step - loss: 0.0723 - acc: 0.9773 - val_loss: 0.0292 - val_acc: 0.9916
Epoch 3/5
30596/30596 [==============================] - 1s 21us/step - loss: 0.0466 - acc: 0.9858 - val_loss: 0.0217 - val_acc: 0.9938
Epoch 4/5
30596/30596 [==============================] - 1s 19us/step - loss: 0.0357 - acc: 0.9894 - val_loss: 0.0145 - val_acc: 0.9949
Epoch 5/5
30596/30596 [==============================] - 1s 19us/step - loss: 0.0298 - acc: 0.9908 - val_loss: 0.0124 - val_acc: 0.9957


In [87]:
score = cnn_1.evaluate(x_test_conv, y_test_class)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

5139/5139 [==============================] - 0s 46us/step
Test loss: 0.012398306225848027
Test accuracy: 0.9957190114808329


### Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

### Make only the dense layers to be trainable and convolutional layers to be non-trainable

In [88]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in cnn_1.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in cnn_1.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv_1
False
max_pooling2d_3
False
conv_2
False
max_pooling2d_4
False
dropout_2
False
flatten_2
False
dense_1
True
dense_2
True


### Do the required preprocessing for `x_train_gt5` also same as `x_train_lt5` and for `y_train_gt5` same as `y_train_lt5`

1. Reshape
2. Change to float32 datatype
3. Normalize (dividing with 255)
4. y_train and y_test Convert into one-hot vectors

In [0]:
x_train_conv = x_train_gt5.reshape(x_train_gt5.shape[0], 28, 28, 1)
x_test_conv = x_test_gt5.reshape(x_test_gt5.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

x_train_conv =  x_train_conv.astype("float32") / 255
x_test_conv = x_test_conv.astype("float32") / 255

y_train_class = keras.utils.to_categorical(y_train_gt5, 5)
y_test_class = keras.utils.to_categorical(y_test_gt5, 5)

In [90]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


In [91]:
cnn_1.fit(x_train_conv, y_train_class, 
                     validation_data=(x_test_conv, y_test_class),        
                     batch_size=512, epochs=5)

Train on 29404 samples, validate on 4861 samples
Epoch 1/5
 9728/29404 [========>.....................] - ETA: 0s - loss: 1.1055 - acc: 0.7184

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


29404/29404 [==============================] - 1s 19us/step - loss: 0.4754 - acc: 0.8705 - val_loss: 0.0887 - val_acc: 0.9702
Epoch 2/5
29404/29404 [==============================] - 1s 20us/step - loss: 0.0861 - acc: 0.9718 - val_loss: 0.0521 - val_acc: 0.9835
Epoch 3/5
29404/29404 [==============================] - 1s 20us/step - loss: 0.0640 - acc: 0.9798 - val_loss: 0.0454 - val_acc: 0.9848
Epoch 4/5
29404/29404 [==============================] - 1s 20us/step - loss: 0.0552 - acc: 0.9826 - val_loss: 0.0339 - val_acc: 0.9879
Epoch 5/5
29404/29404 [==============================] - 1s 19us/step - loss: 0.0467 - acc: 0.9851 - val_loss: 0.0316 - val_acc: 0.9885


In [92]:
score = cnn_1.evaluate(x_test_conv, y_test_class)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

4861/4861 [==============================] - 0s 51us/step
Test loss: 0.031627872225657376
Test accuracy: 0.9884797366796956


# Text classification using TF-IDF

###  Load the dataset from sklearn.datasets

In [0]:

from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [97]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [98]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [99]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [0]:
vect = TfidfVectorizer()
twenty_train_vec = vect.fit_transform(twenty_train.data)

In [102]:
twenty_train_vec.shape

(2257, 35788)

In [103]:
twenty_train_vec.shape

(2257, 35788)

In [0]:
twenty_test_vec = vect.transform(twenty_test.data)

### Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
lg = LogisticRegression()

In [107]:
lg.fit(twenty_train_vec,twenty_train.target)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
twenty_pred_vec = lg.predict(twenty_test_vec)

In [0]:
from sklearn.metrics import accuracy_score

In [110]:
accuracy_score(twenty_test.target, twenty_pred_vec)

0.8868175765645806